# 품질검사성적서 등록 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사성적서 등록 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사성적서 등록 목록/selectIoPmQtscList.csv'

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [3]:
JSONKEY = "items"
DUMMY = 0

In [4]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [5]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv",low_memory=False, encoding="ms949")

In [6]:
dfCombi = imsiDf.loc[:,["cstrnm"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

74882

In [7]:
resultDfMerged = pd.DataFrame()

In [8]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
startYr = 2001
endYr

'2022'

In [9]:
# 기준년도 산정
# 2001년 부터 데이터 존재
# startYr = "2001"
# dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
# # dateRange

## 품질검사성적서 등록정보 전용 스크랩 함수

In [10]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal",x=-1):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount[x] == 0:
                        g_totalCount[x] = int(response.json()["response"]["body"]["totalCount"])
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [11]:
numOfThread = 30

resultDfMerged = list(pd.DataFrame() for x in range(numOfThread))

g_totalCount = list(range(numOfThread))

In [12]:
def threadReq(startIdx, endIdx,x):
    numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
    resultDf = pd.DataFrame()
    for i in range(startIdx,endIdx):
        searchCstrNm = dfCombi.loc[i].cstrnm
        pageNo = 1
        print(searchCstrNm)
        global g_totalCount
        g_totalCount[x] = 0
        while True:
            BASEPARAM={"serviceKey":APIKEY,"searchCstrNm": searchCstrNm, "searchRpcdIsBgDt": str(startYr)+"0101","searchRpcdIsEdDt": str(endYr)+"1231", "sortField": "tstBgDt","pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
            print("BASEPARAM ",BASEPARAM)
            resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY, "jsonabnormal",x)
            if resultDf.empty: # 정상 데이터가 없는 경우
                print("No Data")
                break
            else:
                resultDfMerged[x] = resultDfMerged[x].append(resultDf) 
                g_totalCount[x] -= numOfRows
                print("x : ",x," g_totalCount : ", g_totalCount[x], " numOfRows : ", numOfRows)
                if(g_totalCount[x] <= 0):
                    break
                pageNo += 1

In [13]:
import threading

resultDfLastMerged = pd.DataFrame()
section = list(range(0, dfCombiLen, int(dfCombiLen/numOfThread))) # we have "numåOfThread +1" sections
section

[0,
 2496,
 4992,
 7488,
 9984,
 12480,
 14976,
 17472,
 19968,
 22464,
 24960,
 27456,
 29952,
 32448,
 34944,
 37440,
 39936,
 42432,
 44928,
 47424,
 49920,
 52416,
 54912,
 57408,
 59904,
 62400,
 64896,
 67392,
 69888,
 72384,
 74880]

In [ ]:
# dfCombiLen = 6 # for test
if dfCombiLen < numOfThread:
    print("데이터의 갯수가 적어서 쓰레드를 시작할 수 없습니다.")
else:
    x_ls =list(range(numOfThread))
    thread_list = []
    results = []
    idx = 0
    for x in x_ls:
        startIdx = section[x]
        endIdx =  section[x+1] -1 if (x+1 <= numOfThread -1) else dfCombiLen - 1
        print("start ",startIdx, " end ",endIdx)
        mythread = threading.Thread(target=threadReq, args=(startIdx, endIdx+1,idx))
        idx += 1
        thread_list.append(mythread)
    for i in thread_list:
        i.start()
    for i in thread_list:
        i.join()
    for i in range(0,numOfThread):
        resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 



start  0  end  2495
start  2496  end  4991
start  4992  end  7487
start  7488  end  9983
start  9984  end  12479
start  12480  end  14975
start  14976  end  17471
start  17472  end  19967
start  19968  end  22463
start  22464  end  24959
start  24960  end  27455
start  27456  end  29951
start  29952  end  32447
start  32448  end  34943
start  34944  end  37439
start  37440  end  39935
start  39936  end  42431
start  42432  end  44927
start  44928  end  47423
start  47424  end  49919
start  49920  end  52415
start  52416  end  54911
start  54912  end  57407
start  57408  end  59903
start  59904  end  62399
start  62400  end  64895
start  64896  end  67391
start  67392  end  69887
start  69888  end  72383
start  72384  end  74881
현대건설(주)
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '현대건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
부강건

BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '㈜흥건, ㈜영도종합건설\t\t', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
 {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '풍창건설㈜, ㈜서부종합건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  23  g_totalCount :  -999  numOfRows :  1000
㈜대한중전기
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '㈜대한중전기', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1>totalCount 2
0 no pageNo
x :  16  g_totalCount :  -998  numOfRows :  1000
삼성물산㈜ 건설부분 외 1개사
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5D

>totalCount 10
0 no pageNo
x :  27  g_totalCount :  -990  numOfRows :  1000
금호산업 외 6개사
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '금호산업 외 6개사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 6
0 no pageNo
x :  2  g_totalCount :  -994  numOfRows :  1000
(주)금진건설
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)금진건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 2
0 no pageNo
x :  7  g_totalCount :  -998  numOfRows :  1000
코오롱건설 주식회사
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '코오롱건설 주식회사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page sc

>totalCount 1
0 no pageNo
x :  6  g_totalCount :  -999  numOfRows :  1000
굼호산업 외
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '굼호산업 외', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 44
0 no pageNo
x :  13  g_totalCount :  -956  numOfRows :  1000
KCC건설,롯데건설
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': 'KCC건설,롯데건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 10
0 no pageNo
x :  2  g_totalCount :  -990  numOfRows :  1000
㈜대우건설,㈜명신종합건설
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '㈜대우건설,㈜명신종합건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 pa

>totalCount 4
0 no pageNo
x :  5  g_totalCount :  -996  numOfRows :  1000
sk건설(주)외 2개사
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': 'sk건설(주)외 2개사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  14  g_totalCount :  -999  numOfRows :  1000
대창기업(주)외 1개사
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대창기업(주)외 1개사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  17  g_totalCount :  -999  numOfRows :  1000
(유)플러스건설,이롬건설(주)
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(유)플러스건설,이롬건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000,

...

>totalCount 15
0 no pageNo
x :  10  g_totalCount :  -985  numOfRows :  1000
유촌건성(주)
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '유촌건성(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 2
0 no pageNo
x :  28  g_totalCount :  -998  numOfRows :  1000
장포건설(주)
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '장포건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  18  g_totalCount :  -999  numOfRows :  1000
(주)화승이엔씨
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)화승이엔씨', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping s

>totalCount 1
0 no pageNo
x :  23  g_totalCount :  -999  numOfRows :  1000
미주종합건(주)
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '미주종합건(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 23765
0 no pageNo
x :  0  g_totalCount :  18765  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': 'GS건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 6, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 208
0 no pageNo
x :  3  g_totalCount :  -792  numOfRows :  1000
(유)한샘건설
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(유)한샘건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start

>totalCount 1
0 no pageNo
x :  15  g_totalCount :  -999  numOfRows :  1000
한나래 종합건설(주)
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '한나래 종합건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 2
0 no pageNo
x :  23  g_totalCount :  -998  numOfRows :  1000
주식회사지오콘
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '주식회사지오콘', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4
0 no pageNo
x :  27  g_totalCount :  -996  numOfRows :  1000
주식회사 금남
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '주식회사 금남', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scrap

>totalCount 5
0 no pageNo
x :  25  g_totalCount :  -995  numOfRows :  1000
삼호개발(주)외 3개사
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '삼호개발(주)외 3개사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 30
0 no pageNo
x :  1  g_totalCount :  -970  numOfRows :  1000
(유)남인건설
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(유)남인건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 8
0 no pageNo
x :  14  g_totalCount :  -992  numOfRows :  1000
중흥종합건설㈜, 농업회사법인㈜거유
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '중흥종합건설㈜, 농업회사법인㈜거유', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'typ

>totalCount 1
0 no pageNo
x :  26  g_totalCount :  -999  numOfRows :  1000
해용종합건설㈜											
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '해용종합건설㈜\t\t\t\t\t\t\t\t\t\t\t', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  12  g_totalCount :  -999  numOfRows :  1000
대도물산
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대도물산', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 3>totalCount 3
0 no pageNo
x : 
 20  g_totalCount :  -997  numOfRows :  0 no pageNo1000
(주)성원라인
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)성원라인', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20221231', 'sortField': 'tstBgDt', 'pageNo': 1, 'nu

In [ ]:
resultDfLastMerged

In [ ]:
cf.savedata(resultDfLastMerged, SITENAME,DATANAME,SERVICENAME)

In [ ]:
resultDfLastMerged.shape